In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

SERVER = os.environ.get("SERVER")
DATABASE = os.environ.get("DATABASE")

USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")

conn = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(
    SERVER)+';DATABASE='+str(DATABASE)+';UID='+str(USERNAME)+';PWD=' + str(PASSWORD))
engine = sa.create_engine(
    'mssql+pyodbc:///?odbc_connect={}'.format(conn), fast_executemany=True)
query_smog_synop = """
SELECT *
  FROM [SmogoliczkaArchive].[dbo].[synop_smog]"""
smog_synop = pd.read_sql(query_smog_synop, engine)
smog_synop.head()

,KodStacji,smog_lat,smog_lon,Nazwa,IDStacji,Kraj,synop_lat,synop_lon,distance
0,DsBialka,51.197783,16.117390,Legnica,12415,Poland,51.192571,16.207610,6.313
1,DsBielGrot,50.682510,16.617348,Kłodzko,12520,Poland,50.437235,16.615673,27.274
2,DsBogatFrancMOB,50.940998,14.916790,Zgorzelec,12405,Poland,51.133333,15.033333,22.887
3,DsBogChop,50.905856,14.967175,Zgorzelec,12405,Poland,51.133333,15.033333,25.714
4,DsBogZatonieMob,50.943245,14.913327,Zgorzelec,12405,Poland,51.133333,15.033333,22.741


In [3]:

conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(SERVER)+';DATABASE='+str(DATABASE)+';UID='+str(USERNAME)+';PWD='+ str(PASSWORD))
engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
query_synop="""
SELECT top 1000  d.Stacja as IDStacji_synop,s.Nazwa,s.Kraj, d.Data, d.Godzina, d.PredkoscWiatru
from Depesze d join Stacje as s on d.idstacji=s.IDStacji
  where s.Kraj in ('Poland') and d.Data>='2000-01-01'
  order by d.Data asc
"""
synop_1000=pd.read_sql(query_synop, engine)
synop_1000['Czas pomiaru'] = synop_1000['Data'].apply(pd.to_datetime) + pd.to_timedelta(synop_1000['Godzina'].astype(str), unit="h")
synop_1000.drop(columns=['Data','Godzina'],inplace=True)
synop_1000.head()

,IDStacji_synop,Nazwa,Kraj,PredkoscWiatru,Czas pomiaru
0,12185,Kętrzyn,Poland,2,2000-01-01 05:00:00
1,12195,Suwałki,Poland,0,2000-01-01 05:00:00
2,12120,Łeba,Poland,5,2000-01-01 05:00:00
3,12455,Wieluń,Poland,2,2000-01-01 15:00:00
4,12690,Lesko,Poland,0,2000-01-01 04:00:00


In [4]:

conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(SERVER)+';DATABASE='+str(DATABASE)+';UID='+str(USERNAME)+';PWD='+ str(PASSWORD))
engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
query_smog_pivot="""
SELECT	top 1000 [Kod stacji_metadane]
      ,[Czas pomiaru]
      ,[C6H6]
      ,[CO]
      ,[NO2]
      ,[O3]
      ,[PM10]
      ,[PM2.5]
      ,[SO2]
  FROM [SmogoliczkaArchive].[dbo].[pomiary_pivoted]
  order by [Czas pomiaru] asc
  
"""
smog_pivot=pd.read_sql(query_smog_pivot, engine)
smog_pivot.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2
0,PmGdaWyzwo03,2000-01-01 01:00:00,None,None,NaN,38.0,None,None,None
1,PmGdaPoWie01,2000-01-01 01:00:00,None,None,25.0,NaN,None,None,None
2,MpKrakowWIOSPrad6115,2000-01-01 02:00:00,None,None,38.0,NaN,None,None,None
3,DsSniezkaObs,2000-01-01 02:00:00,None,None,NaN,61.0,None,None,None
4,PmGdaWyzwo03,2000-01-01 02:00:00,None,None,13.0,39.0,None,None,None


In [ ]:
#### Big brain time with merging 

In [5]:
smog_synop.head()

,KodStacji,smog_lat,smog_lon,Nazwa,IDStacji,Kraj,synop_lat,synop_lon,distance
0,DsBialka,51.197783,16.117390,Legnica,12415,Poland,51.192571,16.207610,6.313
1,DsBielGrot,50.682510,16.617348,Kłodzko,12520,Poland,50.437235,16.615673,27.274
2,DsBogatFrancMOB,50.940998,14.916790,Zgorzelec,12405,Poland,51.133333,15.033333,22.887
3,DsBogChop,50.905856,14.967175,Zgorzelec,12405,Poland,51.133333,15.033333,25.714
4,DsBogZatonieMob,50.943245,14.913327,Zgorzelec,12405,Poland,51.133333,15.033333,22.741


In [6]:
synop_1000.tail()

,IDStacji_synop,Nazwa,Kraj,PredkoscWiatru,Czas pomiaru
995,12435,Kalisz,Poland,3,2000-01-01 15:00:00
996,12105,Koszalin,Poland,2,2000-01-01 05:00:00
997,12200,Świnoujście,Poland,2,2000-01-01 05:00:00
998,12160,Elbląg-Milejewo,Poland,2,2000-01-01 05:00:00
999,12497,Włodawa,Poland,3,2000-01-01 15:00:00


In [7]:
smog_pivot.head()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2
0,PmGdaWyzwo03,2000-01-01 01:00:00,None,None,NaN,38.0,None,None,None
1,PmGdaPoWie01,2000-01-01 01:00:00,None,None,25.0,NaN,None,None,None
2,MpKrakowWIOSPrad6115,2000-01-01 02:00:00,None,None,38.0,NaN,None,None,None
3,DsSniezkaObs,2000-01-01 02:00:00,None,None,NaN,61.0,None,None,None
4,PmGdaWyzwo03,2000-01-01 02:00:00,None,None,13.0,39.0,None,None,None


In [ ]:
#SmogoliczkaStacje = pd.merge(SmogoliczkaStacje,SynopStacje,left_on='Nazwa', right_on='Nazwa', how='left')

In [8]:
merge1=pd.merge(synop_1000[['IDStacji_synop','PredkoscWiatru','Czas pomiaru']],smog_synop, left_on='IDStacji_synop',right_on='IDStacji',how='left')
merge1.drop(columns=['smog_lat', 'smog_lon', 'Nazwa', 'IDStacji', 'Kraj', 'synop_lat',
       'synop_lon', 'distance'],inplace=True)
merge2=pd.merge(smog_pivot,merge1, left_on=['Kod stacji_metadane','Czas pomiaru'],right_on=['KodStacji','Czas pomiaru'])
merge2.drop(columns='KodStacji',inplace=True)

In [9]:
merge2.tail()

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji_synop,PredkoscWiatru
168,SlSosnoSosn_narut,2000-01-01 17:00:00,None,None,77.0,NaN,None,None,None,12560,1
169,DsSniezkaObs,2000-01-01 17:00:00,None,None,NaN,66.0,None,None,None,12510,10
170,SlUstroCies_sanat,2000-01-01 17:00:00,None,None,NaN,43.0,None,None,None,12600,2
171,SlKatoKossut,2000-01-01 17:00:00,None,None,NaN,10.0,None,None,None,12560,1
172,PmLebaRabkaE,2000-01-01 18:00:00,None,None,NaN,32.0,None,None,None,12120,3


In [10]:
mask=merge2['PredkoscWiatru']==None

In [11]:
merge2[mask]

,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji_synop,PredkoscWiatru


In [ ]:
#https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.merge



In [12]:
# Bi brain with dask
from dask.diagnostics import ProgressBar
ProgressBar().register()


conn = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(
    SERVER)+';DATABASE='+str(DATABASE)+';UID='+str(USERNAME)+';PWD=' + str(PASSWORD))
engine = sa.create_engine(
    'mssql+pyodbc:///?odbc_connect={}'.format(conn), fast_executemany=True)
query_synop_dask = """
SELECT top 100000 d.Stacja as IDStacji_synop,s.Nazwa,s.Kraj, d.Data, d.Godzina
		      ,d.[KierunekWiatru]
      ,d.[PredkoscWiatru]
      ,d.[TemperaturaPowietrza]
      ,d.[TemperaturaPunktuRosy]
      ,d.[CisnienieNaPoziomieStacji]
      ,d.[CisnienieNaPoziomieMorza]
      ,d.[WielkoscTendencjiCisnienia]
      ,d.[WysokoscOpadu]
      ,d.[OkresPomiaruOpadu]
from Depesze d join Stacje as s on d.idstacji=s.IDStacji
  where s.Kraj in ('Poland') and d.Data>='2000-01-01'
  order by d.Data asc
"""
ddf = dd.from_pandas(pd.read_sql(query_synop_dask, engine), npartitions=20)


def f(df: pd.DataFrame) -> pd.DataFrame:
    """ This is your code from above, as a function """
    df['Czas pomiaru'] = df['Data'].apply(
        pd.to_datetime) + pd.to_timedelta(df['Godzina'].astype(str), unit="h")
    # df.drop(columns=['Godzina','Nazwa','Kraj'],inplace=True)
    return df


columns = names = ["IDStacji_synop", 'Nazwa', 'Kraj', 'Data', 'Godzina', "KierunekWiatru",
                   "PredkoscWiatru", "TemperaturaPowietrza", "TemperaturaPunktuRosy",
                   "CisnienieNaPoziomieStacji", "CisnienieNaPoziomieMorza",
                   "WielkoscTendencjiCisnienia", "WysokoscOpadu", "OkresPomiaruOpadu", "Czas pomiaru"
                   ]
df = pd.DataFrame(columns=columns)
ddf = ddf.map_partitions(f, meta=df)
ddf['IDStacji_synop'] = ddf['IDStacji_synop'].astype('int64')
ddf['Czas pomiaru'] = ddf['Czas pomiaru'].astype('datetime64')

In [ ]:
# ddf = dd.read_csv('./raw_data/filtered/synop02.02.csv',sep=';',decimal=',',names=["IDStacji_synop","Nazwa","Kraj","Data","Godzina","KierunekWiatru",
#        "PredkoscWiatru","TemperaturaPowietrza","TemperaturaPunktuRosy",
#        "CisnienieNaPoziomieStacji","CisnienieNaPoziomieMorza",
#        "WielkoscTendencjiCisnienia","WysokoscOpadu","OkresPomiaruOpadu"
# ],assume_missing=True)

In [13]:
##

conn = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(
    SERVER)+';DATABASE='+str(DATABASE)+';UID='+str(USERNAME)+';PWD=' + str(PASSWORD))
engine = sa.create_engine(
    'mssql+pyodbc:///?odbc_connect={}'.format(conn), fast_executemany=True)
query_synop_smog_pomiary_pivoted = """
  select top 10000 pp.[Kod stacji_metadane]
      ,pp.[Czas pomiaru]
      ,pp.[C6H6]
      ,pp.[CO]
      ,pp.[NO2]
      ,pp.[O3]
      ,pp.[PM10]
      ,pp.[PM2.5]
      ,pp.[SO2]
	  ,ss.IDStacji
  from pomiary_pivoted as pp join synop_smog as ss on pp.[Kod stacji_metadane]=ss.KodStacji
  order by pp.[Czas pomiaru] asc
  """
ddf2 = dd.from_pandas(pd.read_sql(
    query_synop_smog_pomiary_pivoted, engine), npartitions=20)

In [14]:
ddf.dtypes

IDStacji_synop                         int64
Nazwa                                 object
Kraj                                  object
Data                                  object
Godzina                               object
KierunekWiatru                        object
PredkoscWiatru                        object
TemperaturaPowietrza                  object
TemperaturaPunktuRosy                 object
CisnienieNaPoziomieStacji             object
CisnienieNaPoziomieMorza              object
WielkoscTendencjiCisnienia            object
WysokoscOpadu                         object
OkresPomiaruOpadu                     object
Czas pomiaru                  datetime64[ns]
dtype: object

In [15]:
ddf2.dtypes

Kod stacji_metadane            object
Czas pomiaru           datetime64[ns]
C6H6                           object
CO                             object
NO2                           float64
O3                            float64
PM10                           object
PM2.5                          object
SO2                            object
IDStacji                        int64
dtype: object

In [16]:
ddf3=ddf2.merge(ddf, left_on=['IDStacji','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')

In [3]:
ddf3.head()

NameError: name 'ddf3' is not defined

In [20]:
ddf3.tail(50)

[########################################] | 100% Completed | 18.0s


,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji,IDStacji_synop,Nazwa,Kraj,Data,Godzina,KierunekWiatru,PredkoscWiatru,TemperaturaPowietrza,TemperaturaPunktuRosy,CisnienieNaPoziomieStacji,CisnienieNaPoziomieMorza,WielkoscTendencjiCisnienia,WysokoscOpadu,OkresPomiaruOpadu
452,SlBytomByto_modrz,2000-01-16 20:00:00,None,None,43.0,NaN,None,None,None,12560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453,LbJarczWolaM,2000-01-17 00:00:00,None,None,NaN,46.0,None,None,None,12490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,MzBelsIGFPAN,2000-01-17 02:00:00,None,None,9.0,36.0,None,None,None,12471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455,SlUstroCies_sanat,2000-01-17 02:00:00,None,None,NaN,61.0,None,None,None,12600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,MpKrakAlKras,2000-01-17 02:00:00,None,None,30.0,NaN,None,None,None,12566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,WmPuszczaBor,2000-01-17 02:00:00,None,None,NaN,46.0,None,None,None,12185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
458,PmTczeTarg07,2000-01-17 03:00:00,None,None,7.0,NaN,None,None,None,12155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
459,DsJelw05,2000-01-17 05:00:00,None,None,10.0,46.0,None,None,None,12405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
460,DsCzerStraza,2000-01-17 05:00:00,None,None,11.0,30.0,None,None,None,12405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461,WmPuszczaBor,2000-01-17 05:00:00,None,None,NaN,56.0,None,None,None,12185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test1.head()

In [ ]:
test1.shape

In [ ]:
test2.head()

In [ ]:
test3=pd.merge(test2,test1, left_on=['IDStacji','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')

In [ ]:
test3.columns

In [ ]:
test3.head()

In [2]:
import coiled

In [3]:
# # Connects to the existing "production" cluster
cluster = coiled.Cluster(
    name="test"
)

Using existing cluster: test


In [4]:
from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

Dashboard: http://ec2-18-191-47-203.us-east-2.compute.amazonaws.com:8787


C:\Users\p_sierkin\Miniconda3\envs\smogoliczka\lib\site-packages\distributed\client.py:1135: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| blosc   | None          | 1.10.2        | 1.10.2        |
| lz4     | None          | 3.1.3         | 3.1.3         |
| numpy   | 1.20.1        | 1.20.0        | 1.20.0        |
| python  | 3.8.0.final.0 | 3.8.6.final.0 | 3.8.6.final.0 |
+---------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [5]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [32]:
### end product
#### Big brain with dask
# from dask.diagnostics import ProgressBar
# ProgressBar().register()

# conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(server)+';DATABASE='+str(database)+';UID='+str(username)+';PWD='+ str(password))
# engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
# query_synop_dask ="""
# SELECT d.Stacja as IDStacji_synop,s.Nazwa,s.Kraj, d.Data, d.Godzina
# 		      ,d.[KierunekWiatru]
#       ,d.[PredkoscWiatru]
#       ,d.[TemperaturaPowietrza]
#       ,d.[TemperaturaPunktuRosy]
#       ,d.[CisnienieNaPoziomieStacji]
#       ,d.[CisnienieNaPoziomieMorza]
#       ,d.[WielkoscTendencjiCisnienia]
#       ,d.[WysokoscOpadu]
#       ,d.[OkresPomiaruOpadu]
# from Depesze d join Stacje as s on d.idstacji=s.IDStacji
#   where s.Kraj in ('Germany','Poland','Belarus','Czech Republic','Lithuania','Slovakia','Ukraine') and d.Data>='2000-01-01'
#   order by d.Data asc
# """
#ddf = dd.from_pandas(pd.read_sql(query_synop_dask, engine), npartitions=20)


ddf = dd.read_csv('file://data/synop_02.02.csv',sep=';',decimal=',',assume_missing=True
)

# ,names=["IDStacji_synop","Nazwa","Kraj","Data","Godzina","KierunekWiatru",
#        "PredkoscWiatru","TemperaturaPowietrza","TemperaturaPunktuRosy",
#        "CisnienieNaPoziomieStacji","CisnienieNaPoziomieMorza",
#        "WielkoscTendencjiCisnienia","WysokoscOpadu","OkresPomiaruOpadu"
# ]


In [8]:
import pandas as pd
df = pd.read_csv('data/synop_02.02.csv',sep=';',decimal=',')


In [9]:
future = client.scatter(df)  # send dataframe to one worker


In [11]:
import dask.dataframe as dd

ddf = dd.from_delayed([future], meta=df)  # build dask.dataframe on remote data



In [12]:
ddf = ddf.repartition(npartitions=20).persist()  # split

In [13]:
client.rebalance(ddf)  # spread around all of your workers

In [58]:
#client.upload_file('./data/synop_02.02.csv')

In [14]:
ddf=ddf[ddf['Kraj']=='Poland']

In [15]:
ddf.head()

,IDStacji_synop,Nazwa,Kraj,Data,Godzina,KierunekWiatru,PredkoscWiatru,TemperaturaPowietrza,TemperaturaPunktuRosy,CisnienieNaPoziomieStacji,CisnienieNaPoziomieMorza,WielkoscTendencjiCisnienia,WysokoscOpadu,OkresPomiaruOpadu
0,12295,Białystok,Poland,2000-01-01,16:00:00,270.0,2,0.0,-1.3,1004.7,1023.9,0.3,0.0,0
1,12280,Mikołajki,Poland,2000-01-01,16:00:00,230.0,1,-0.1,-0.9,1006.9,1023.6,0.1,0.0,0
2,12272,Olsztyn,Poland,2000-01-01,16:00:00,220.0,1,-0.7,-1.8,1006.3,1023.7,0.1,0.0,0
3,12270,Mława,Poland,2000-01-01,16:00:00,240.0,3,-1.7,-2.0,1005.4,1024.4,0.0,0.0,0
4,12250,Toruń,Poland,2000-01-01,16:00:00,150.0,2,-1.0,-2.1,1014.5,1023.7,0.4,0.0,0


In [16]:
ddf

,IDStacji_synop,Nazwa,Kraj,Data,Godzina,KierunekWiatru,PredkoscWiatru,TemperaturaPowietrza,TemperaturaPunktuRosy,CisnienieNaPoziomieStacji,CisnienieNaPoziomieMorza,WielkoscTendencjiCisnienia,WysokoscOpadu,OkresPomiaruOpadu
npartitions=20,,,,,,,,,,,,,,
,int64,object,object,object,object,float64,int64,object,object,object,object,object,object,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:

def f(df: pd.DataFrame) -> pd.DataFrame:
    """ This is your code from above, as a function """
    df['Czas pomiaru'] = df['Data'].apply(pd.to_datetime) + pd.to_timedelta(df['Godzina'].astype(str), unit="h")
    #df.drop(columns=['Godzina','Nazwa','Kraj'],inplace=True)
    return df

columns = names=["IDStacji_synop",'Nazwa','Kraj','Data','Godzina',"KierunekWiatru",
       "PredkoscWiatru","TemperaturaPowietrza","TemperaturaPunktuRosy",
       "CisnienieNaPoziomieStacji","CisnienieNaPoziomieMorza",
       "WielkoscTendencjiCisnienia","WysokoscOpadu","OkresPomiaruOpadu","Czas pomiaru"
]
df = pd.DataFrame(columns=columns)
ddf=ddf.map_partitions(f,meta=df)
ddf['IDStacji_synop']=ddf['IDStacji_synop'].astype('int64')
ddf['Czas pomiaru']=ddf['Czas pomiaru'].astype('datetime64')


In [21]:
ddf.tail()

ValueError: unit must not be specified if the input contains a str

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [20]:
ddf.dtypes

IDStacji_synop                         int64
Nazwa                                 object
Kraj                                  object
Data                                  object
Godzina                               object
KierunekWiatru                        object
PredkoscWiatru                        object
TemperaturaPowietrza                  object
TemperaturaPunktuRosy                 object
CisnienieNaPoziomieStacji             object
CisnienieNaPoziomieMorza              object
WielkoscTendencjiCisnienia            object
WysokoscOpadu                         object
OkresPomiaruOpadu                     object
Czas pomiaru                  datetime64[ns]
dtype: object

In [17]:
# ##

# conn= urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+str(server)+';DATABASE='+str(database)+';UID='+str(username)+';PWD='+ str(password))
# engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conn),fast_executemany=True)
# query_synop_smog_pomiary_pivoted= """
#   select pp.[Kod stacji_metadane]
#       ,pp.[Czas pomiaru]
#       ,pp.[C6H6]
#       ,pp.[CO]
#       ,pp.[NO2]
#       ,pp.[O3]
#       ,pp.[PM10]
#       ,pp.[PM2.5]
#       ,pp.[SO2]
# 	  ,ss.IDStacji
#   from pomiary_pivoted as pp join synop_smog as ss on pp.[Kod stacji_metadane]=ss.KodStacji
#   order by pp.[Czas pomiaru] asc
#   """
# ddf2 = dd.from_pandas(pd.read_sql(query_synop_smog_pomiary_pivoted, engine), npartitions=20)
ddf2 = dd.read_csv('./data/smogoliczkaarchive_02.02.csv',
                   sep=';', decimal=',', assume_missing=True)

In [18]:
ddf.head()

FileNotFoundError: [Errno 2] No such file or directory: 'E:/OneDrive - Warszawska Wyższa Szkoła Informatyki/inzynierka/data/synop_02.02.csv'

In [19]:
ddf2.head()

FileNotFoundError: [Errno 2] No such file or directory: 'E:/OneDrive - Warszawska Wyższa Szkoła Informatyki/inzynierka/data/smogoliczkaarchive_02.02.csv'

In [20]:
ddf2['IDStacji']=ddf2['IDStacji'].astype('int64')
ddf2['Czas pomiaru']=ddf2['Czas pomiaru'].astype('datetime64')

In [21]:
ddf3=ddf2.merge(ddf, left_on=['IDStacji','Czas pomiaru'],right_on=['IDStacji_synop','Czas pomiaru'],how='left')

In [22]:
ddf3.shape

(Delayed('int-5cd55550-d327-495e-b332-d5862cec9897'), 24)

In [48]:
ddf3.to_csv('/data/all/synopsmogdane.csv', single_file = True)

[                                        ] | 0% Completed |  5min 36.6s


KeyboardInterrupt: 

In [50]:
ddf3.to_parquet('./data/all/test/')


[########################################] | 100% Completed | 30min 35.4s


In [51]:
test = dd.read_parquet('./data/all/test/', engine='pyarrow')


In [52]:
test.head()

[########################################] | 100% Completed |  1.8s


,Kod stacji_metadane,Czas pomiaru,C6H6,CO,NO2,O3,PM10,PM2.5,SO2,IDStacji,IDStacji_synop,Nazwa,Kraj,Data,Godzina,KierunekWiatru,PredkoscWiatru,TemperaturaPowietrza,TemperaturaPunktuRosy,CisnienieNaPoziomieStacji,CisnienieNaPoziomieMorza,WielkoscTendencjiCisnienia,WysokoscOpadu,OkresPomiaruOpadu
0,DsJasnaSport,2003-05-02 13:00:00,NaN,NaN,10.0,NaN,NaN,NaN,21.0,11603,NaN,None,None,None,None,NaN,NaN,None,None,None,None,None,None,NaN
1,DsCzerStraza,2003-05-02 13:00:00,NaN,NaN,NaN,NaN,4.0,NaN,2.0,11603,NaN,None,None,None,None,NaN,NaN,None,None,None,None,None,None,NaN
2,KpWloclWSHE,2003-05-02 14:00:00,NaN,NaN,NaN,11.9,NaN,NaN,NaN,12360,12360.0,Płock,Poland,2003-05-02,14:00:00,230.0,2.0,19.0,5.2,1004.0,1016.8,1.4,0.0,0.0
3,KpWloclOpsis,2003-05-02 14:00:00,NaN,NaN,9.8,NaN,NaN,NaN,3.4,12360,12360.0,Płock,Poland,2003-05-02,14:00:00,230.0,2.0,19.0,5.2,1004.0,1016.8,1.4,0.0,0.0
4,MzPlockPKN3,2003-05-02 14:00:00,NaN,0.31,6.9,94.0,NaN,NaN,1.0,12360,12360.0,Płock,Poland,2003-05-02,14:00:00,230.0,2.0,19.0,5.2,1004.0,1016.8,1.4,0.0,0.0


In [53]:
print("Number of rows => {:,d}".format(len(test)))
print("Number of columns => {}".format(len(test.columns)))

[########################################] | 100% Completed |  3.2s
Number of rows => 23,880,736
Number of columns => 24


In [58]:
len(test[test['IDStacji_synop']==None])

[########################################] | 100% Completed | 27.9s


0

In [55]:
from loader_sql import PandasToSQL

In [56]:
PandasToSQL().dask_loader(test,"pomiary_pivoted_synop")

  0%|          | 0/30 [00:00<?, ?it/s]

[########################################] | 100% Completed |  1.7s


  3%|▎         | 1/30 [06:27<3:07:15, 387.43s/it]

[########################################] | 100% Completed |  1.6s


  7%|▋         | 2/30 [09:55<2:35:45, 333.76s/it]

...

 93%|█████████▎| 28/30 [1:17:29<05:18, 159.33s/it]

[########################################] | 100% Completed |  1.7s


 97%|█████████▋| 29/30 [1:20:22<02:43, 163.36s/it]

[########################################] | 100% Completed |  2.2s


100%|██████████| 30/30 [1:22:56<00:00, 165.89s/it]
